In [2]:
import os
import cv2
import numpy as np

In [9]:

image_folder = "../Simulation-mask-paired-data/val/B"
mask_folder = "../Simulation-mask-paired-data/val/B-masks/"
output_folder = "../Simulation-mask-paired-data/arm-green/val/B/"

os.makedirs(output_folder, exist_ok=True)
image_filenames = sorted([f for f in os.listdir(image_folder) if f.endswith('.png')])

for filename in image_filenames:
    image_path = os.path.join(image_folder, filename)
    mask_path = os.path.join(mask_folder, filename)
    
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) 

    if image is None or mask is None:
        print(f"Skipping {filename}, missing image or mask.")
        continue

    green_background = np.full_like(image, (0, 255, 0)) 

    object_mask = (mask > 0).astype(np.uint8)

    object_mask_3ch = cv2.merge([object_mask]*3)

    result = np.where(object_mask_3ch == 1, image, green_background)

    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, result)
    print(f"Saved: {output_path}")


Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_1004.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_1009.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_103.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_113.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_117.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_124.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_138.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_157.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_162.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_164.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_168.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_174.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_179.png
Saved: ../Simulation-mask-paired-data/arm-green/val/B/image_180.png
Saved: ../Simulation-mask-paired-data/arm-gree